# Import Libraries 

In [1]:
#Selenium - Chrome Driver to Scrape Web Pages 
#Beautiful Soup - Web Scrapper 
#Requests - Get/Post informaton fetching using URLs
#Pandas - For Data handling and analysis
#pymongo - Database

from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import pymongo
from urllib.request import urlopen

# Selenium Chrome Driver Set Up
### Two Coders with different Paths to the Executable file 

## Yamini's Driver Path

In [2]:
#Commented when used by Pratham Doshi
driver= webdriver.Chrome('C:/Users/renuka/Desktop/chromedriver.exe')

## Pratham's Driver Path

In [3]:
#Commented when used by Yamini
#driver = webdriver.Chrome('/Users/prathamdoshi/Desktop/chromedriver')

# Fetching Information 

In [3]:
driver.get("https://pluginamerica.org/vehicles/?fwp_type=ev")
plugin_america_website_soup = BeautifulSoup(driver.page_source,"lxml")
driver.quit()

# Get Car Model/Make

In [35]:
#html object that has all the car models listed
car_models_list_html = plugin_america_website_soup.find_all('td', class_="make_model sorting_1")

#Extracting the company name and the model out of the html object
car_make_model = []
car_company = []

for model_make in car_models_list_html:
    model_make_text_string = model_make.text
    model_make_string_splitter = model_make_text_string.split(' ',1)
    #print(model_make_string_splitter)
    car_company.append(model_make_string_splitter[0])
    car_make_model.append(model_make_text_string)

In [33]:
car_make_model

['BMW i3',
 'Chevrolet Bolt EV',
 'Fiat 500e',
 'Ford Focus Electric',
 'Honda Clarity Electric',
 'Hyundai Ioniq Electric',
 'Kia Soul EV',
 'Mercedes B250e',
 'Nissan LEAF',
 'Quantya Strada',
 'Smart ED',
 'Tesla Model 3',
 'Tesla Model S',
 'Tesla Model X',
 'Victory Empulse TT',
 'Volkswagen e-Golf',
 'Zero Motorcycles Zero DS/DSR',
 'Zero Motorcycles Zero FX/FXS',
 'Zero Motorcycles Zero S/SR']

# Get Car Description

In [9]:
#html object that has all the car descriptions listed
car_description_list_html = plugin_america_website_soup.find_all('td', class_="car_description")
#Extracting the car description out of the html object
cars_descriptions = []

for description in car_description_list_html:
    description_text_string = description.p.text
    cars_descriptions.append(description_text_string)

# Get Car Range

In [42]:
#html object that has all the car ranges listed
car_range_list_html = plugin_america_website_soup.find_all('td', class_="car_range")
#Extracting the car ranges out of the html object
cars_ranges = []

for range_ in car_range_list_html:
    range_text_string = range_.p.text
    #print(range_text_string)
    range_text_string = range_text_string.rstrip().replace('\t', '')
    print(range_text_string)
    #.replace('\t', '')
    cars_ranges.append(range_text_string)


 114 mi
 238 mi
 87 mi
 115 mi
 89 mi
 124 mi
 111 mi
 87 mi
 151 mi
 30 mi
 68 mi
 220 mi - 310 mi
 210 mi - 335 mi
 237 mi - 295 mi
 130 mi
 125 mi
 110 mi - 147 mi
 41 mi - 90 mi
 121 mi - 161 mi


In [27]:
cars_ranges

[' 114 mi',
 ' 238 mi',
 ' 87 mi',
 ' 115 mi',
 ' 89 mi',
 ' 124 mi',
 ' 111 mi',
 ' 87 mi',
 ' 151 mi',
 ' 30 mi',
 ' 68 mi',
 ' 220 mi - 310 mi',
 ' 210 mi - 335 mi',
 ' 237 mi - 295 mi',
 ' 130 mi',
 ' 125 mi',
 ' 110 mi - 147 mi',
 ' 41 mi - 90 mi',
 ' 121 mi - 161 mi']

# Get Number of Seats

In [11]:
#html object that has number of seats in a car listed
car_seats_list_html = plugin_america_website_soup.find_all('td', class_="number_of_seats")
#Extracting the car seats out of the html object
cars_no_of_seats = []

for seats in car_seats_list_html:
    seats_text_string = seats.p.text
    cars_no_of_seats.append(seats_text_string)

# Get Price

In [12]:
#html object that has car prices listed
car_prices_list_html = plugin_america_website_soup.find_all('td', class_="car_price")
#Extracting the car seats out of the html object
cars_price = []

for price in car_prices_list_html:
    price_text_string = price.p.text
    price_text_string = price_text_string.rstrip().replace('\t', '')
    price_text_string = price_text_string.rstrip().replace('  ', '')
    cars_price.append(price_text_string)

# Get Images

In [13]:
#html object that has car images URL listed
car_images_list_html = plugin_america_website_soup.find_all('td', class_="car_img")
cars_images_url = []

for url in car_images_list_html:
    cars_images_url.append((url.a.attrs['href']))

# Object to Return

In [26]:
cars_information = []
cars_information_appender = dict()
for i in range(len(car_company)):
    cars_information_appender = {
                                 car_make_model[i] : {
                                                 'Company':car_company[i],
                                                 'Image Url':cars_images_url[i],
                                                 'Description':cars_descriptions[i],
                                                 'Price':cars_price[i],
                                                 'Range':cars_ranges[i],
                                                 'Number of Seats':cars_no_of_seats[i]                          
                                                } 
                                }
    cars_information.append(cars_information_appender)
    #print(cars_information_appender)

In [23]:
cars_information_appender

{'Zero Motorcycles Zero S/SR': {'Company': 'Zero',
  'Description': 'More power. Higher top speed. Greater acceleration. The 2017 Zero S and SR Series motorcycles feature more torque than most 1000cc sport bikes. The S Series offers up to 161 miles of range, a 0-60 time of 5.2 seconds, and a top speed of 95 miles per hour. The SR…',
  'Image Url': 'https://pluginamerica.org/plug-in-vehicle/zero-motorcycles-zero-ssr/',
  'Number of Seats': '1',
  'Price': ' $9,895 - $14,395',
  'Range': ' 121 mi - 161 mi'}}

# Database Integration
## Database Name : Cars_DB, Collection Name: Cars

In [12]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.Cars_DB
collection = db.Cars

#collection insert
collection.insert_many(cars_information)